# Setup

In [347]:
import logging
import sys
import os
import openai
from IPython.display import Markdown, display
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/karlkeat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [389]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# Set OpenAI API Key
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Add helicone API key
openai.api_base = "http://oai.hconeai.com/v1"
headers = {
            "Helicone-Auth": f"Bearer {os.environ.get('HELICONE_API_KEY')}",
            "Helicone-Cache-Enabled": "true"
          }

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", api_base="http://oai.hconeai.com/v1", api_key=os.environ["OPENAI_API_KEY"], default_headers=headers)
embed_model = OpenAIEmbedding(temperature=0.1, model="text-embedding-3-small", api_base="http://oai.hconeai.com/v1", api_key=os.environ["OPENAI_API_KEY"], default_headers=headers)

Settings.llm = llm
Settings.embed_model = embed_model

# Build Clinical Guidelines Retriever

In [391]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)

In [392]:
# Run during first time setup only
cpic_guideline_documents = SimpleDirectoryReader("./cpic_guidelines").load_data()
cpic_guideline_index = VectorStoreIndex.from_documents(cpic_guideline_documents)
cpic_guideline_index.set_index_id("cpic_guidelines_vector_index")
cpic_guideline_index.storage_context.persist("./vector_index_storage")

INFO:httpx:HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP 

In [393]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="vector_index_storage")
# load index
cpic_guideline_index = load_index_from_storage(storage_context, index_id="cpic_guidelines_vector_index")

INFO:llama_index.core.indices.loading:Loading indices with ids: ['cpic_guidelines_vector_index']
Loading indices with ids: ['cpic_guidelines_vector_index']
Loading indices with ids: ['cpic_guidelines_vector_index']
Loading indices with ids: ['cpic_guidelines_vector_index']
Loading indices with ids: ['cpic_guidelines_vector_index']
Loading indices with ids: ['cpic_guidelines_vector_index']


In [404]:
cpic_guideline_retriever = cpic_guideline_index.as_retriever(similarity_top_k=5)
cpic_guideline_nodes = cpic_guideline_retriever.retrieve("CYP2C19*1/*2, clopidogrel?")
cpic_guideline_nodes

INFO:httpx:HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='ada3a8d8-6a4f-48aa-9f31-e0b2e8327c8d', embedding=None, metadata={'page_label': '318', 'file_name': '23698643.pdf', 'file_path': '/home/karlkeat/projects/PGx-LLM-RAG/cpic_guidelines/23698643.pdf', 'file_type': 'application/pdf', 'file_size': 250786, 'creation_date': '2024-04-02', 'last_modified_date': '2024-04-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4eecab71-b9ff-4ae7-8aab-cc19d8cbb949', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '318', 'file_name': '23698643.pdf', 'file_path': '/home/karlkeat/projects/PGx-LLM-RAG/cpic_guidelines/23698643.pdf', 'file_type': 'application/pdf', 'file_size': 250786, 'creation_date': '2024-04-02', 'last_modified_

In [401]:
def get_cpic_guidelines(query_str, top_k=5):
    cpic_guideline_retriever = cpic_guideline_index.as_retriever(similarity_top_k=top_k)
    return cpic_guideline_retriever.retrieve(query_str)

# CPIC Database Setup

In [397]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    text
)

In [356]:
engine = create_engine("postgresql://karlkeat@localhost:5432/cpic_db_rag", # Replace with your own db path
                       connect_args={'options': '-csearch_path=cpic'})
metadata_obj = MetaData()
metadata_obj.create_all(engine)
metadata_obj.reflect(engine)
table_names = list(metadata_obj.tables.keys())
table_names

['gene',
 'pair',
 'drug',
 'guideline',
 'test_alert',
 'allele_definition',
 'allele_frequency',
 'allele',
 'population',
 'publication',
 'change_log',
 'recommendation',
 'gene_result',
 'gene_result_lookup',
 'gene_result_diplotype',
 'file_artifact',
 'flyway_schema_history',
 'file_note',
 'sequence_location',
 'term',
 'statistic',
 'publication_supplement',
 'allele_location_value',
 'file_artifact_history']

In [357]:
sql_database = SQLDatabase(engine)

## Database query test

In [358]:
stmt = "select d.genesymbol, d.name, sl.dbsnpid\nfrom allele_definition d\njoin allele_location_value alv on d.id = alv.alleledefinitionid\njoin sequence_location sl on alv.locationid = sl.id\nwhere d.genesymbol='CYP2C19' and d.name='*2';"
with engine.connect() as connection:
    results = connection.execute(text(stmt)).fetchall()
    print(results)

[('CYP2C19', '*2', 'rs12769205'), ('CYP2C19', '*2', 'rs58973490'), ('CYP2C19', '*2', 'rs4244285'), ('CYP2C19', '*2', 'rs3758581')]


# Text-to-SQL Query Engine

In [359]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import NLSQLRetriever

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name=table)) for table in table_names
]  # add a SQLTableSchema for each table
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [369]:
with open("./cpic_db_docs/cpic_wiki.md") as infile:
    text = infile.read()
    
cpic_db_wiki = {}
for section in text.split("###")[1:]:
    header = section.split("\n")[0].strip().lower().replace(" ", "_")
    body = "\n".join(section.split("\n")[2:]).strip()
    cpic_db_wiki[header] = body
cpic_db_wiki

{'gene': "- table: `gene`\n\nThis model represents a human gene and is in the table `gene`. It includes properties that are specific to the gene \nlike references to other data sources such as NCBI, HGNC, and Ensembl.\n\nThe primary key for a Gene is the `symbol` property. This is the official HGNC symbol for that gene.\n\nThe presence of a gene in this model does NOT guarantee that a CPIC guideline exists for that gene.\n\nAs an example, here's how you would get information about CYP2D6.\n\n\nselect * from gene where symbol='CYP2D6';",
 'drug': '- table: `drug`\n\nThe "drug" model represents a drug referenced somewhere in the data model. It includes references to other drug data \nsources like ATC, RxNorm, and UMLS.\n\nThe primary key for a Drug is the `drugid` property. The value of this property is in the form "source:id". For \nexample: `RxNorm:2670` for codeine. We attempt to use RxNorm as the primary key for most drugs, but some CPIC drug \nentities don\'t have equivalent values 

In [402]:
def run_text_to_sql(text_to_sql_query):
    # Fetch relevant SQL table documentation
    table_retriever = obj_index.as_retriever(similarity_top_k=5)
    tables_to_query = [x.table_name for x in table_retriever.retrieve(text_to_sql_query)]

    context = ""

    # Add wiki docs to context
    for table in tables_to_query:
        if table in cpic_db_wiki:
            context += cpic_db_wiki[table]

    if len(context) > 0:
        context = "Here is some useful information on the SQL table structure: \n" + context + "\n"

    # Query text2sql model
    nl_sql_retriever = NLSQLRetriever(
        sql_database, return_raw=True, tables = tables_to_query, handle_sql_errors=True
    )
    return nl_sql_retriever.retrieve_with_metadata(context + text_to_sql_query)

In [362]:
results = run_text_to_sql("What are the dbsnpids (not pharmvarIDs) associated with CYP2C19 *2?")
results

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'pair' has columns: pairid (INTEGER): 'A synthetic numerical id, automatically assigned, primary key', genesymbol (VARCHAR(20)): 'The HGNC symbol of the gene in this pair, required', drugid (VARCHAR(20)): 'The ID of the drug in this pair, required', guidelineid (INTEGER): 'The ID of a guideline this pair is described in, optional', usedforrecommendation (BOOLEAN): 'Whether the gene is used for recommendation lookup for the drug if this pair is part of a guideline, default false', version (INTEGER): 'The version number, iterates on modification', cpiclevel (VARCHAR(5)): 'The CPIC level of this pair, requir

([NodeWithScore(node=TextNode(id_='17585b84-39ab-4a4f-be99-d78db6cec36f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[('CYP2C19', '*2', 'rs12769205'), ('CYP2C19', '*2', 'rs58973490'), ('CYP2C19', '*2', 'rs4244285'), ('CYP2C19', '*2', 'rs3758581')]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)],
 {'sql_query': "select d.genesymbol, d.name, sl.dbsnpid\nfrom allele_definition d\njoin allele_location_value alv on d.id = alv.alleledefinitionid\njoin sequence_location sl on alv.locationid = sl.id\nwhere d.genesymbol='CYP2C19' and d.name='*2';",
  'result': [('CYP2C19', '*2', 'rs12769205'),
   ('CYP2C19', '*2', 'rs58973490'),
   ('CYP2C19', '*2', 'rs4244285'),
   ('CYP2C19', '*2', 'rs3758581')],
  'col_keys': ['genesymbol', 'name', 'dbsnpid']})

In [363]:
results = run_text_to_sql("What is the clinical functionality of the CYP2C19*3 allele?")
results

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'allele' has columns: id (INTEGER): 'A synthetic numerical ID, auto-assigned, primary key', version (INTEGER): 'The version number, iterates on modification', genesymbol (VARCHAR(50)): 'The HGNC symbol of the gene the allele is for, required', name (TEXT): 'The name of this allele, required', functionalstatus (TEXT): 'The functional phenotype of this allele', clinicalfunctionalstatus (TEXT): 'The functional phenotype of this allele used for clinical systems', clinicalfunctionalsubstrate (TEXT): 'Allele clinical function substrate specificity, optional', activityvalue (VARCHAR(50)): 'Descriptor of activity

([NodeWithScore(node=TextNode(id_='28553aaa-6340-4269-9f31-fd48c1cfef1d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(None,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)],
 {'sql_query': "SELECT a.functionalstatus\nFROM allele a\nJOIN allele_definition ad ON a.definitionid = ad.id\nWHERE ad.genesymbol = 'CYP2C19' AND ad.name = '*3';",
  'result': [(None,)],
  'col_keys': ['functionalstatus']})

In [373]:
results = run_text_to_sql("What is the frequency weighted average of CYP2C19*2 in the European group?")
results

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'allele_frequency' has columns: alleleid (INTEGER): 'The ID of the allele for this observation', population (INTEGER): 'The ID of the population for this observation', frequency (NUMERIC): 'The numeric representation of this frequency', label (TEXT): 'The textual label for this frequency, "-" means no observation made', version (INTEGER): 'The version number, iterates on modification', with comment: (A frequency observation for a particular allele in a particular population) and foreign keys: ['alleleid'] -> allele.['id'], ['population'] -> population.['id'].

Table 'allele' has columns: id (INTEGER): 'A 

([NodeWithScore(node=TextNode(id_='5814f13f-23ee-40b5-a749-9d4b75a2de58', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[(Decimal('0.14685704098527318700'),)]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)],
 {'sql_query': "SELECT freq_weighted_avg \nFROM population_frequency_view \nWHERE genesymbol = 'CYP2C19' \nAND population_group = 'European' \nAND name = '*2';",
  'result': [(Decimal('0.14685704098527318700'),)],
  'col_keys': ['freq_weighted_avg']})

In [376]:
results = run_text_to_sql("Should someone with CYP2C19*2/*2 take clopidogrel?")
results

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'drug' has columns: drugid (VARCHAR(20)): 'A unique identifier for this drug in the form "source:id" where source is some outside knowledge resource and id is their identifier, primary key', name (TEXT): 'The generic name for this drug, lower-cased, required', pharmgkbid (VARCHAR(20)): 'The PharmGKB ID for this drug, optional', rxnormid (VARCHAR(20)): 'The RxNorm ID for this drug, optional', drugbankid (VARCHAR(20)): 'The DrugBank ID for this drug, optional', atcid (ARRAY): 'One or more ATC IDs for this drug in an array, optional', umlscui (VARCHAR(20)): 'The UMLS Concept Unique ID for this drug, optional

([NodeWithScore(node=TextNode(id_='71ede359-66ca-4fee-9703-7372177889e8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)],
 {'sql_query': 'select * from recommendation where lookupKey=\'{"CYP2C19": "*2/*2"}\' and drugid=\'RxNorm:1205\';',
  'result': [],
  'col_keys': ['id',
   'guidelineid',
   'drugid',
   'implications',
   'drugrecommendation',
   'classification',
   'phenotypes',
   'activityscore',
   'allelestatus',
   'lookupkey',
   'population',
   'comments',
   'version',
   'dosinginformation',
   'alternatedrugavailable',
   'otherprescribingguidance']})

# Joint SQL + Guidelines LLM

In [423]:
def joint_query(query_str):
    # Call CPIC vector DB model to retreive relevant guideline snippets
    guidelines = get_cpic_guidelines(query_str, top_k=3)
    guidelines = "\n".join([x.node.text for x in guidelines])

    # Call text2sql model to retreive relevant DB output
    sql_results = run_text_to_sql(query_str)[1]
    if "result" in sql_results:
        sql_results = sql_results["result"]
    else:
        sql_results = ""

    # Attach context to query and submit
    prompt = f"Here is some useful context to answer this query, do not use any outside information: {guidelines}\n{sql_results}\nHere is the query: {query_str}"
    return llm.complete(prompt)

In [426]:
result = joint_query("What are the SNP IDs associated with CYP2C19 *17?")
result

INFO:httpx:HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://oai.hconeai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'allele' 

CompletionResponse(text='The SNP IDs associated with CYP2C19 *17 are rs12248560 and rs3758581.', additional_kwargs={}, raw={'id': 'chatcmpl-9A2XonaOD90Gj40dpFqcxKSRrvavi', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The SNP IDs associated with CYP2C19 *17 are rs12248560 and rs3758581.', role='assistant', function_call=None, tool_calls=None))], 'created': 1712180088, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': 'fp_b28b39ffa8', 'usage': CompletionUsage(completion_tokens=23, prompt_tokens=2295, total_tokens=2318)}, logprobs=None, delta=None)

In [427]:
result.text

'The SNP IDs associated with CYP2C19 *17 are rs12248560 and rs3758581.'